<a href="https://colab.research.google.com/github/jjangmo91/Cervus-nippon_Anmado-Is./blob/main/GF_SG_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 30.8 MB/s eta 0:00:00


In [20]:
# 필요한 모듈 임포트
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from osgeo import gdal
import requests
import json
import geopandas as gpd
import os
import shutil
import psutil
from google.colab import drive
import zipfile
import glob
import rasterio
from rasterio import mask
import numpy.ma as ma  # 마스킹된 배열을 처리하기 위한 모듈

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91')

Google Drive에 저장된 데이터를 가져옵니다.

In [21]:
# Google Drive 마운트
drive.mount('/content/drive')

# 데이터 폴더 경로 설정
data_dir = '/content/drive/MyDrive/Deer2/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# TIFF 파일 목록 가져오기
tiff_files = glob.glob(os.path.join(data_dir, '*.tif'))

# 파일명에서 연도 추출하여 데이터프레임 생성
file_data = []

for file_path in tiff_files:
    # 파일명 추출
    file_name = os.path.basename(file_path)

    # 파일명에서 연도 추출 (예: 'synseries_sg_anmado_2014-0000000000-0000000000.tif'에서 '2014' 추출)
    try:
        year_str = file_name.split('_')[2]  # 파일명에서 세 번째 요소가 연도임 ('2014')
        year = int(year_str)  # 연도가 숫자로 변환 가능한지 확인
    except (IndexError, ValueError):
        print(f"Error extracting year from file name: {file_name}")
        continue

    # 파일 경로와 연도 정보를 저장
    file_data.append({'file_path': file_path, 'year': year})

# 데이터프레임 생성
if file_data:
    files_df = pd.DataFrame(file_data)

    # 연도 순으로 정렬
    files_df = files_df.sort_values(by='year').reset_index(drop=True)

    # 결과 출력
    print(files_df)
else:
    print("No valid TIFF files found with extractable year information.")

Error extracting year from file name: synseries_sg_anmado_2015-0000000000-0000000000.tif
Error extracting year from file name: synseries_sg_anmado_2014-0000000000-0000000000.tif
Error extracting year from file name: synseries_sg_anmado_2016-0000000000-0000000000.tif
Error extracting year from file name: synseries_sg_anmado_2017-0000000000-0000000000.tif
Error extracting year from file name: synseries_sg_anmado_2018-0000000000-0000000000.tif
Error extracting year from file name: synseries_sg_anmado_2019-0000000000-0000000000.tif
Error extracting year from file name: synseries_sg_anmado_2020-0000000000-0000000000.tif
No valid TIFF files found with extractable year information.


In [34]:
# GeoJSON 파일을 GEE FeatureCollection으로 변환하는 함수
def geojson_to_feature_collection(url):
    response = requests.get(url)
    geojson = response.json()
    features = []
    for feature in geojson['features']:
        geom = feature['geometry']
        feat = {'type': 'Feature', 'geometry': geom, 'properties': feature['properties']}
        features.append(feat)
    return features

# Anmado 및 Songido FeatureCollection 생성
url_anmado = 'https://github.com/jjangmo91/Cervus-nippon_Anmado-Is./raw/main/anmado.geojson'
url_songido = 'https://github.com/jjangmo91/Cervus-nippon_Anmado-Is./raw/main/songido.geojson'

anmado_features = geojson_to_feature_collection(url_anmado)
songido_features = geojson_to_feature_collection(url_songido)

# GeoDataFrame으로 변환
anmado_gdf = gpd.GeoDataFrame.from_features(anmado_features)
songido_gdf = gpd.GeoDataFrame.from_features(songido_features)

# 각 지역에 대한 AOI 설정 함수
def set_aoi_by_frtp(features, region_name):
    frtp_values = ['0', '1', '2', '3']
    aoi_dict = {}

    for frtp_value in frtp_values:
        # 해당 지역에서 FRTP_CD 값을 가진 Feature 추출
        filtered_features = [feat for feat in features if feat['properties']['FRTP_CD'] == frtp_value]

        if len(filtered_features) == 0:
            print(f"{region_name}: FRTP_CD {frtp_value}에 대한 Feature가 없습니다.")
            continue

        # GeoPandas를 사용하여 Feature를 GeoDataFrame으로 변환
        gdf = gpd.GeoDataFrame.from_features(filtered_features)

        # AOI 설정 (MultiPolygon으로 결합)
        aoi = gdf.unary_union
        aoi_dict[frtp_value] = aoi

        print(f"{region_name}: FRTP_CD {frtp_value}에 대한 AOI가 설정되었습니다.")

    return aoi_dict

래스터 데이터의 CRS를 AOI 좌표계와 일치시켜줍니다.

In [35]:
# Anmado와 Songido에 대한 AOI 설정
anmado_aoi_dict = set_aoi_by_frtp(anmado_features, "Anmado")
songido_aoi_dict = set_aoi_by_frtp(songido_features, "Songido")

# AOI의 기본 좌표계를 설정 (GeoJSON 파일의 기본 좌표계는 EPSG:4326일 가능성이 높음)
anmado_gdf = gpd.GeoDataFrame(geometry=anmado_gdf.geometry, crs='EPSG:4326')
songido_gdf = gpd.GeoDataFrame(geometry=songido_gdf.geometry, crs='EPSG:4326')

# 래스터 데이터의 CRS 설정 (예: EPSG:32652)
raster_crs = 'EPSG:32652'

# AOI의 좌표계를 래스터 데이터의 CRS로 변환
anmado_gdf = anmado_gdf.to_crs(raster_crs)
songido_gdf = songido_gdf.to_crs(raster_crs)

# 변환된 CRS 출력
print(f"Transformed Anmado AOI CRS: {anmado_gdf.crs}")
print(f"Transformed Songido AOI CRS: {songido_gdf.crs}")

<ipython-input-34-f2fb1240462a>:40: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  aoi = gdf.unary_union


Anmado: FRTP_CD 0에 대한 AOI가 설정되었습니다.
Anmado: FRTP_CD 1에 대한 AOI가 설정되었습니다.
Anmado: FRTP_CD 2에 대한 AOI가 설정되었습니다.
Anmado: FRTP_CD 3에 대한 AOI가 설정되었습니다.
Songido: FRTP_CD 0에 대한 AOI가 설정되었습니다.
Songido: FRTP_CD 1에 대한 AOI가 설정되었습니다.
Songido: FRTP_CD 2에 대한 AOI가 설정되었습니다.
Songido: FRTP_CD 3에 대한 AOI가 설정되었습니다.
Transformed Anmado AOI CRS: EPSG:32652
Transformed Songido AOI CRS: EPSG:32652


데이터에 이미 GF-SG가 적용되었으므로, 대상지역의 NDVI 시계열 데이터를 추출하고 시각화합니다.

In [36]:
# 결과를 저장할 딕셔너리 초기화 (Anmado와 Songido를 각각 처리)
frtp_values = ['0', '1', '2', '3']
ndvi_data_by_frtp_anmado = {frtp_value: [] for frtp_value in frtp_values}
ndvi_data_by_frtp_songido = {frtp_value: [] for frtp_value in frtp_values}

# 밴드가 이미 시간순으로 정렬되어 있다고 가정
if not files_df.empty:
    for idx, row in files_df.iterrows():
        file_path = row['file_path']
        year = row['year']  # 파일명에서 추출한 연도 사용

        # TIFF 파일 열기
        with rasterio.open(file_path) as src:
            band_count = src.count  # 밴드 수 확인
            raster_crs = src.crs  # 래스터의 CRS 가져오기

            # 각 지역 (Anmado와 Songido)에 대해 AOI를 래스터의 CRS로 변환
            transformed_aois_anmado = {}
            transformed_aois_songido = {}

            for frtp_value in frtp_values:
                # Anmado AOI 변환
                if frtp_value in anmado_aoi_dict:
                    aoi_anmado = anmado_aoi_dict[frtp_value]
                    aoi_gdf_anmado = gpd.GeoDataFrame(geometry=[aoi_anmado], crs='EPSG:4326')
                    aoi_gdf_anmado = aoi_gdf_anmado.to_crs(raster_crs)
                    transformed_aois_anmado[frtp_value] = aoi_gdf_anmado.iloc[0].geometry

                # Songido AOI 변환
                if frtp_value in songido_aoi_dict:
                    aoi_songido = songido_aoi_dict[frtp_value]
                    aoi_gdf_songido = gpd.GeoDataFrame(geometry=[aoi_songido], crs='EPSG:4326')
                    aoi_gdf_songido = aoi_gdf_songido.to_crs(raster_crs)
                    transformed_aois_songido[frtp_value] = aoi_gdf_songido.iloc[0].geometry

            # 각 밴드별로 처리 (밴드가 시간 순으로 이미 정렬되어 있음)
            for band_idx in range(1, band_count + 1):
                # Anmado에 대한 NDVI 평균값 계산
                for frtp_value in frtp_values:
                    if frtp_value in transformed_aois_anmado:
                        aoi_anmado = transformed_aois_anmado[frtp_value]
                        try:
                            ndvi_masked, _ = mask.mask(
                                src,
                                [aoi_anmado],
                                crop=True,
                                all_touched=True,
                                indexes=band_idx,
                                filled=False  # filled를 False로 설정하여 마스크된 배열 반환
                            )
                            ndvi_masked = ndvi_masked[0]  # 첫 번째 배열 선택
                            ndvi_masked = ndvi_masked.astype('float32') / 10000.0  # NDVI 값 보정

                            # 누락값(-9999)을 마스킹 처리
                            ndvi_masked = np.ma.masked_where(ndvi_masked == -9999, ndvi_masked)

                            # NDVI 평균값 계산 (마스크된 값을 무시)
                            ndvi_mean = ndvi_masked.mean()
                        except ValueError as e:
                            print(f"Anmado ValueError for FRTP_CD {frtp_value}: {e}")
                            ndvi_mean = np.nan

                        # 결과 저장 (밴드 번호와 연도를 기반으로 저장)
                        ndvi_data_by_frtp_anmado[frtp_value].append({
                            'year': year,
                            'band': band_idx,
                            'NDVI': ndvi_mean
                        })

                # Songido에 대한 NDVI 평균값 계산
                for frtp_value in frtp_values:
                    if frtp_value in transformed_aois_songido:
                        aoi_songido = transformed_aois_songido[frtp_value]
                        try:
                            ndvi_masked, _ = mask.mask(
                                src,
                                [aoi_songido],
                                crop=True,
                                all_touched=True,
                                indexes=band_idx,
                                filled=False  # filled를 False로 설정하여 마스크된 배열 반환
                            )
                            ndvi_masked = ndvi_masked[0]  # 첫 번째 배열 선택
                            ndvi_masked = ndvi_masked.astype('float32') / 10000.0  # NDVI 값 보정

                            # 누락값(-9999)을 마스킹 처리
                            ndvi_masked = np.ma.masked_where(ndvi_masked == -9999, ndvi_masked)

                            # NDVI 평균값 계산 (마스크된 값을 무시)
                            ndvi_mean = ndvi_masked.mean()
                        except ValueError as e:
                            print(f"Songido ValueError for FRTP_CD {frtp_value}: {e}")
                            ndvi_mean = np.nan

                        # 결과 저장 (밴드 번호와 연도를 기반으로 저장)
                        ndvi_data_by_frtp_songido[frtp_value].append({
                            'year': year,
                            'band': band_idx,
                            'NDVI': ndvi_mean
                        })

In [38]:
# 데이터프레임 생성
ndvi_df_list = []

# Anmado와 Songido의 데이터를 결합하여 데이터프레임 생성
for region_name, ndvi_data_by_frtp in zip(['Anmado', 'Songido'], [ndvi_data_by_frtp_anmado, ndvi_data_by_frtp_songido]):
    for frtp_value, data_list in ndvi_data_by_frtp.items():
        df = pd.DataFrame(data_list)
        if not df.empty:
            df['FRTP_CD'] = frtp_value
            df['Region'] = region_name
            ndvi_df_list.append(df)

# 모든 데이터를 하나의 데이터프레임으로 결합
if ndvi_df_list:
    ndvi_df = pd.concat(ndvi_df_list)

    # 날짜 형식 변환 및 정렬 (연도를 datetime 형식으로 변환)
    ndvi_df['date'] = pd.to_datetime(ndvi_df['year'], format='%Y')
    ndvi_df = ndvi_df.sort_values(by='date')

    # FRTP_CD를 카테고리형으로 변환하고 순서 지정
    frtp_order = ['0', '1', '2', '3']
    ndvi_df['FRTP_CD'] = pd.Categorical(ndvi_df['FRTP_CD'], categories=frtp_order, ordered=True)

    # 시각화 스타일 설정
    sns.set(style="whitegrid")

    # 시계열 그래프 그리기
    plt.figure(figsize=(14, 7))
    sns.lineplot(
        data=ndvi_df,
        x='date',
        y='NDVI',
        hue='FRTP_CD',
        style='Region',
        hue_order=frtp_order,  # 순서 지정
        marker='o',
        palette='tab10'
    )

    # 그래프 제목 및 레이블 설정
    plt.title('NDVI Time Series Comparison by FRTP and Region', fontsize=16)
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Mean NDVI Value', fontsize=14)
    plt.legend(title='FRTP_CD and Region', fontsize=12, title_fontsize=13)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No valid NDVI data available for plotting.")

No valid NDVI data available for plotting.
